In [1]:
import sys
import os
import re
import string
import numpy as np
import math
import nltk
import io
import csv
import pandas as pd
import trax
import textwrap
wrapper = textwrap.TextWrapper(width=70)


from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from itertools import islice
from rouge import Rouge
from trax import layers as tlayer
from trax.fastmath import numpy as trax_np
from trax.supervised import training



2023-06-07 09:18:03.313177: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# nltk.download('stopwords')
nltk.download('stopwords')  # uncomment these lines once they are not downloaded
nltk.download('punkt')  # uncomment these lines once they are not downloaded

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mohammedzaki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/mohammedzaki/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Extractive Summary

In [2]:
def preprocessing(article):
    
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')

    article_preprocessed = []
    sentences = sent_tokenize(article)
    for sentence in sentences:
        sentence_preprocessed = []
        sentence = re.sub(r"[^a-zA-Z\s]+", "", sentence)
        words = word_tokenize(sentence)
        for word in words:
            if (word not in stopwords_english and word not in string.punctuation):
                word_stemmed = stemmer.stem(word)  
                sentence_preprocessed.append(word_stemmed)
        if sentence_preprocessed:
            article_preprocessed.append(sentence_preprocessed)
            
    return article_preprocessed


In [3]:
def convert_list_to_string(sentences):  # converts list of lists to list of strings
    sentences_modified = []   # list of strings
    for sentence in sentences:
        sentence_modified = ' '.join(sentence)
        sentences_modified.append(sentence_modified)
    return sentences_modified

In [4]:
### Feature 1

def calculate_TF_IDF(content):
    flat_words = [word for sent in content for word in sent]
    words_set = set(flat_words)
    words_num = len(words_set)
    tf = pd.DataFrame(np.zeros((len(content), words_num)), columns = list(words_set))
    for i in range (len(content)):
        for w in content[i]:
                      tf[w][i] += 1/len(content[i])
                      
    idf = {}
    
    for word in words_set:
        num_docs = 0
        for i in range(len(content)):
            if word in content[i]:
                num_docs += 1
                
        idf[word] = np.log10(len(content) / num_docs)
        
    tf_idf = np.zeros(len(content))
    
    for i in range (len(content)):
        for word in content[i]:
            tf_idf[i] += tf[word][i] * idf[word]
            
    tf_idf = tf_idf/max(tf_idf)  # might be commented (this normalizes the tf-idf)
            
    return tf_idf

In [5]:
### Feature 2

def sentence_length(article_preprocessed):
    article_preprocessed = convert_list_to_string(article_preprocessed)
    max_length = 0
    for sentence in article_preprocessed:
        if len(sentence.split()) > max_length:
            max_length = len(sentence.split())
            
    sentence_length_feature = []
    for sentence in article_preprocessed:
        sentence_length_feature.append(len(sentence.split()) / max_length)


    return sentence_length_feature

In [6]:
### Feature 3

def proper_nouns_number(sentences):
    number_proper_nouns_in_sentences = []
    proper_nouns_number_feature = []
    stopwords_english = stopwords.words('english')
    for sentence in sentences:
        words_in_sentence = re.sub(r"[^a-zA-Z\s]+", "", sentence)
        words = word_tokenize(words_in_sentence)
        count_true_words = 0
        for word in words:
            if (word not in stopwords_english and word not in string.punctuation):
                count_true_words += 1
        if(count_true_words > 0):
            tokens = nltk.word_tokenize(sentence)
            tagged = nltk.pos_tag(tokens)
            proper_nouns = [word for word, pos in tagged if pos == 'NNP']
            number_proper_nouns_in_sentences.append(len(proper_nouns))
#         print(proper_nouns)
        
    if max(number_proper_nouns_in_sentences) > 0:
        for number_proper_nouns_in_sentence in number_proper_nouns_in_sentences:
            proper_nouns_number_feature.append(number_proper_nouns_in_sentence / max(number_proper_nouns_in_sentences))
    else:
        for number_proper_nouns_in_sentence in number_proper_nouns_in_sentences:
            proper_nouns_number_feature.append(0)

    return proper_nouns_number_feature

In [7]:
def generate_X_labels(article_preprocessed, original_sentences):
    # feature 1 (tf_idf)
    tf_idf_score_feature = calculate_TF_IDF(article_preprocessed)
    
    # feature 2 (sentence_length)
    sentence_length_feature = sentence_length(article_preprocessed)
    
    # feature 3 (proper nouns number)
    num_proper_nouns_feature = proper_nouns_number(original_sentences)
    
    matrix = np.column_stack((tf_idf_score_feature, sentence_length_feature, num_proper_nouns_feature))

    return matrix

In [8]:
def generate_Y_labels(original, summarized):
    Y_list = []
    original_sentences = sent_tokenize(original)
    original_sentences[0] = original_sentences[0][1:] # to remove the \n
    summarized_sentences = sent_tokenize(summarized)
    
    for original_sentence in original_sentences:
        added = 0
        for summarized_sentence in summarized_sentences:
            if original_sentence in summarized_sentence:
                Y_list.append(1)
                added = 1
                break
        if added == 0:
            Y_list.append(0)
    
    return Y_list, original_sentences
    

In [9]:
X_matrix = []
X = []
Y = []
sentences = []

article_types = ['business', 'entertainment', 'politics', 'sport', 'tech']

for article_type in article_types:
    for i in range (1, 51):   # loading business articles
        article_file = io.open("train_original/" + article_type + "/article (" + str(i) +").txt", "r", encoding='utf8')
        article_file.readline()
        article = article_file.read()
        article_file.close()

        summarized_file = io.open("train_summary/" + article_type + "/summary (" + str(i) +").txt", "r", encoding='utf8')
        summarized = summarized_file.read()
        summarized_file.close()

        article_preprocessed = preprocessing(article)
        original_sentences = sent_tokenize(article)
        original_sentences[0] = original_sentences[0][1:] # to remove the \n
        X_i = generate_X_labels(article_preprocessed, original_sentences)
        Y_i, original_list_no_first_space = generate_Y_labels(article, summarized)

        if(len(X_i) != len(Y_i)):
            print('Error! features and labels are not equal in length')

        Y.extend(Y_i)
        X_matrix.extend(X_i)
        sentences.extend(original_list_no_first_space)
    

for x in X_matrix:
    X.append(x.tolist())
    
X = np.matrix(X)

m = len(X)

print(len(X))
print(len(Y))

4179
4179


In [10]:
nn_input_dim = 3 # input layer size (we have two input features)
nn_output_dim = 1  # output layer size (we have one output)

# Gradient descent parameters
alpha = 0.1  # learning rate for gradient descent

In [11]:
def sigmoid(x):
    sig = (1/(1+np.exp(-x)))
    return sig

def softmax(x):
    e_x = np.exp(x - np.max(x))  # Subtracting the maximum value for numerical stability
    return e_x / np.sum(e_x, axis=0)

In [12]:
def build_extractive_model(nn_hdim, num_passes=20000, print_loss=False):
    np.random.seed(0)
    W1 = np.random.randn(nn_hdim, nn_input_dim) / np.sqrt(nn_input_dim)
    b1 = np.zeros((nn_hdim, 1))
    W2 = np.random.randn(nn_hdim, nn_hdim) / np.sqrt(nn_input_dim)
    b2 = np.zeros((nn_hdim, 1))
    W3 = np.random.randn(nn_hdim, nn_hdim) / np.sqrt(nn_input_dim)
    b3 = np.zeros((nn_hdim, 1))
    W4 = np.random.randn(nn_hdim, nn_hdim) / np.sqrt(nn_input_dim)
    b4 = np.zeros((nn_hdim, 1))
    W5 = np.random.randn(nn_hdim, nn_hdim) / np.sqrt(nn_input_dim)
    b5 = np.zeros((nn_hdim, 1))
    W6 = np.random.randn(nn_hdim, nn_hdim) / np.sqrt(nn_input_dim)
    b6 = np.zeros((nn_hdim, 1))
    W7 = np.random.randn(nn_output_dim, nn_hdim) / np.sqrt(nn_hdim)
    b7 = np.zeros((nn_output_dim, 1))

    model = {}

    for i in range(0, num_passes):
        DW1 = 0
        DW2 = 0
        DW3 = 0
        DW4 = 0
        DW5 = 0
        DW6 = 0
        DW7 = 0
        
        Db1 = 0
        Db2 = 0
        Db3 = 0
        Db4 = 0
        Db5 = 0
        Db6 = 0
        Db7 = 0
        cost = 0
        
        for j in range(0, m):
            a0 = X[j, :].reshape(-1, 1)  # Every training example is a column vector.
            y = Y[j]
            
            z1 = np.dot(W1 , a0)+ b1
            a1 = np.tanh(z1)
            z2 = np.dot(W2 , a1)+ b2
            a2 = np.tanh(z2)
            z3 = np.dot(W3 , a2)+ b3
            a3 = np.tanh(z3)
            z4 = np.dot(W4 , a3)+ b4
            a4 = np.tanh(z4)
            z5 = np.dot(W5 , a4)+ b5
            a5 = np.tanh(z5)
            z6 = np.dot(W6 , a5)+ b6
            a6 = np.tanh(z6)
            z7 = np.dot(W7 , a6) + b7
            a7 = sigmoid(z7)

            cost_j = -1 * ((np.log(a7) * y + (1-y)* np.log(1-a7)))

            dz7 = a7 - y
            dW7 = np.dot(dz7 , a6.T)
            db7 = dz7
            
            da6 = np.dot(W7.T, dz7)
            dz6 = np.multiply(da6 , 1 - np.square(a6) )
            dW6 = np.dot(dz6 , a5.T )
            db6 = dz6
            
            da5 = np.dot(W6.T, dz6)
            dz5 = np.multiply(da5 , 1 - np.square(a5) )
            dW5 = np.dot(dz5 , a4.T )
            db5 = dz5
            
            da4 = np.dot(W5.T, dz5)
            dz4 = np.multiply(da4 , 1 - np.square(a4) )
            dW4 = np.dot(dz4 , a3.T )
            db4 = dz4
            
            da3 = np.dot(W4.T, dz4)
            dz3 = np.multiply(da3 , 1 - np.square(a3) )
            dW3 = np.dot(dz3 , a2.T )
            db3 = dz3
            
            da2 = np.dot(W3.T, dz3)
            dz2 = np.multiply(da2 , 1 - np.square(a2) )
            dW2 = np.dot(dz2 , a1.T )
            db2 = dz2

            da1 = np.dot(W2.T, dz2)
            dz1 = np.multiply(da1 , 1 - np.square(a1) )
            dW1 = np.dot(dz1 , a0.T )
            db1 = dz1
            
            DW1 += dW1
            DW2 += dW2
            DW3 += dW3
            DW4 += dW4
            DW5 += dW5
            DW6 += dW6
            DW7 += dW7

            Db1 += db1
            Db2 += db2
            Db3 += db3
            Db4 += db4
            Db5 += db5
            Db6 += db6
            Db7 += db7
            
            cost += cost_j
        
        DW1 /= m
        DW2 /= m
        DW3 /= m
        DW4 /= m
        DW5 /= m
        DW6 /= m
        DW7 /= m
        
        Db1 /= m
        Db2 /= m
        Db3 /= m
        Db4 /= m
        Db5 /= m
        Db6 /= m
        Db7 /= m
        
        cost /= m

        W1 -= alpha * DW1
        b1 -= alpha * Db1
        W2 -= alpha * DW2
        b2 -= alpha * Db2
        W3 -= alpha * DW3
        b3 -= alpha * Db3
        W4 -= alpha * DW4
        b4 -= alpha * Db4
        W5 -= alpha * DW5
        b5 -= alpha * Db5
        W6 -= alpha * DW6
        b6 -= alpha * Db6
        W7 -= alpha * DW7
        b7 -= alpha * Db7

        model = {'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2, 'W3': W3, 'b3': b3, 'W4': W4, 'b4': b4,
                 'W5': W5, 'b5': b5, 'W6': W6, 'b6': b6, 'W7': W7, 'b7': b7}

        if print_loss and i % 1000 == 0:
            print("Loss after iteration %i: %f" % (i, cost))

    return model


In [13]:
def predict_extractive_model(model, x):
    W1, b1, W2, b2, W3, b3, W4, b4, W5, b5, W6, b6, W7, b7 = model['W1'], model['b1'], model['W2'], model['b2'], model['W3'], model['b3'], + \
                                                             model['W4'], model['b4'], model['W5'], model['b5'], model['W6'], model['b6'], + \
                                                             model['W7'], model['b7']
    a0 = x.T

    z1 = np.dot(W1 , a0)+ b1
    a1 = np.tanh(z1)
    z2 = np.dot(W2 , a1)+ b2
    a2 = np.tanh(z2)
    z3 = np.dot(W3 , a2)+ b3
    a3 = np.tanh(z3)
    z4 = np.dot(W4 , a3)+ b4
    a4 = np.tanh(z4)
    z5 = np.dot(W5 , a4)+ b5
    a5 = np.tanh(z5)
    z6 = np.dot(W6 , a5)+ b6
    a6 = np.tanh(z6)
    z7 = np.dot(W7 , a6) + b7
    a7 = sigmoid(z7)

    prediction = a7    
    return prediction

In [16]:
model = build_extractive_model(nn_hdim= 3, num_passes = 10001, print_loss=True)

Loss after iteration 0: 0.705638
Loss after iteration 1000: 0.571912
Loss after iteration 2000: 0.565582
Loss after iteration 3000: 0.564406
Loss after iteration 4000: 0.562721
Loss after iteration 5000: 0.559442
Loss after iteration 6000: 0.557169
Loss after iteration 7000: 0.556274
Loss after iteration 8000: 0.556055
Loss after iteration 9000: 0.555855
Loss after iteration 10000: 0.555651


In [17]:
print(model)

{'W1': array([[ 0.69843821, -0.19993617,  0.36440881],
       [ 0.95903598,  1.42450063,  0.66612429],
       [ 2.1827463 , -0.08856085, -0.19113287]]), 'b1': array([[-0.66530508],
       [-0.60750625],
       [-1.03570939]]), 'W2': array([[ 0.65695835, -0.449375  ,  1.70663831],
       [ 0.29069578,  0.2822306 ,  0.27918986],
       [ 0.31508882,  1.00816188, -0.34445394]]), 'b2': array([[-0.43456939],
       [ 0.00045761],
       [-0.31112202]]), 'W3': array([[ 0.11780398, -0.51736596, -1.56001077],
       [ 0.84980301,  0.65826252, -0.46672083],
       [ 1.52883098, -0.7284255 , -0.17640248]]), 'b3': array([[ 0.18916633],
       [ 0.06281344],
       [-0.16717293]]), 'W4': array([[ 0.0919177 ,  1.14554835,  1.04559518],
       [-0.33379555,  0.21200468, -0.67899995],
       [-1.20565714, -0.17297097,  0.09055869]]), 'b4': array([[ 0.02975375],
       [ 0.35624101],
       [-0.16862802]]), 'W5': array([[ 1.26675989,  0.59541044, -0.36713659],
       [-0.16337134, -0.59990016, -0.8375

In [26]:
model = {'W1': np.array([[ 0.69843821, -0.19993617,  0.36440881],
       [ 0.95903598,  1.42450063,  0.66612429],
       [ 2.1827463 , -0.08856085, -0.19113287]]), 'b1': np.array([[-0.66530508],
       [-0.60750625],
       [-1.03570939]]), 'W2': np.array([[ 0.65695835, -0.449375  ,  1.70663831],
       [ 0.29069578,  0.2822306 ,  0.27918986],
       [ 0.31508882,  1.00816188, -0.34445394]]), 'b2': np.array([[-0.43456939],
       [ 0.00045761],
       [-0.31112202]]), 'W3': np.array([[ 0.11780398, -0.51736596, -1.56001077],
       [ 0.84980301,  0.65826252, -0.46672083],
       [ 1.52883098, -0.7284255 , -0.17640248]]), 'b3': np.array([[ 0.18916633],
       [ 0.06281344],
       [-0.16717293]]), 'W4': np.array([[ 0.0919177 ,  1.14554835,  1.04559518],
       [-0.33379555,  0.21200468, -0.67899995],
       [-1.20565714, -0.17297097,  0.09055869]]), 'b4': np.array([[ 0.02975375],
       [ 0.35624101],
       [-0.16862802]]), 'W5': np.array([[ 1.26675989,  0.59541044, -0.36713659],
       [-0.16337134, -0.59990016, -0.83756652],
       [-1.15320666,  1.3348449 ,  0.0362139 ]]), 'b5': np.array([[0.30658246],
       [0.16403235],
       [0.27393751]]), 'W6': np.array([[-0.34851762, -0.75102519,  0.69490768],
       [-1.33427527, -0.18975495, -0.14022591],
       [ 0.42502826, -0.18174999, -1.10706824]]), 'b6': np.array([[-0.23081771],
       [-0.1860524 ],
       [ 0.05209766]]), 'W7': np.array([[ 0.87815907,  0.80358377, -1.31396283]]), 'b7': np.array([[-0.92787612]])}

In [18]:
def test(article_preprocessed_test, original_test, summarized_text, compression_ratio, original_sentences, file_number = 0):
    X_test = generate_X_labels(article_preprocessed_test, original_sentences)
    predicton = predict_extractive_model(model, X_test)
    Y_test, original_sentences = generate_Y_labels(original_test, summarized_text)
    num_sentences_summarized = math.ceil(compression_ratio * len(original_sentences))
    
    
    highest = np.argsort(predicton[0]) [::-1]
    highest = highest[: num_sentences_summarized]
#     highest = sorted(highest) # uncomment to arrange the article
    output_sentences = []
    output_indices = []
    

    
    for i in range (0, num_sentences_summarized):
        output_sentences.append(original_sentences[highest[i]])
        output_indices.append(highest[i])
        
    output_sentences = ''.join(output_sentences)
    
    rouge = Rouge()
#     rouge = Rouge(metrics=['rouge-n', 'rouge-l'], max_n=2)
    scores_nn = rouge.get_scores(output_sentences, summarized_text)
    scores_nn = scores_nn[0]
    
    rouge_1_nn = scores_nn['rouge-1']['f']
    rouge_2_nn = scores_nn['rouge-2']['f']
    rouge_l_nn = scores_nn['rouge-l']['f']
    
    
    
    ### classified using TF_idf score

    output_tf_idf = summary_using_tf_idf_only(original_test, num_sentences_summarized)
    
    scores_tf_idf = rouge.get_scores(output_tf_idf, summarized_text)
    scores_tf_idf = scores_tf_idf[0]
    
    rouge_1_tf_idf = scores_tf_idf['rouge-1']['f']
    rouge_2_tf_idf = scores_tf_idf['rouge-2']['f']
    rouge_l_tf_idf = scores_tf_idf['rouge-l']['f']

    
    return rouge_1_nn, rouge_2_nn, rouge_l_nn, rouge_1_tf_idf, rouge_2_tf_idf, rouge_l_tf_idf

In [19]:
def summary_using_tf_idf_only(text, num_sentences):
    article_sentences = sent_tokenize(text)
    article_preprocessed = preprocessing(text)
    sentence_scores = calculate_TF_IDF(article_preprocessed)
    highest = np.argsort(sentence_scores) [::-1]
    highest = highest[: num_sentences]
    highest = sorted(highest) # uncomment to arrange the article
    output_sentences = []
    
    for i in range (0, num_sentences):
        output_sentences.append(article_sentences[highest[i]])
        
    
    output_sentences = ' '.join(output_sentences)
    
    return output_sentences

In [20]:
article_file = io.open("cr7.txt", "r", encoding='utf8')
article = article_file.read()
article_file.close
summary = summary_using_tf_idf_only(article, 7)
print(summary)

Real Madrid forward Cristiano Ronaldo has said that he is the "best player in history" after winning his fifth Ballon d'Or on Thursday. Ronaldo picked up the award for the second year in a row to equal the record of Barcelona star Lionel Messi, and he said he does not believe any player is better than him. He told France Football (h/t Goal's Robin Bairner): "I've never seen anyone better than me. I play well with both feet, I’m quick, powerful, good with the head, I score goals, I make assists. That says something, doesn’t it? Legends like Floyd Mayweather [Jr.] and LeBron James don’t get to their perfect level by chance. To be at the top and to stay there, you have to have more talent than the others."


In [21]:
# testing on BBC news summary

rouge_1_list_nn = []
rouge_2_list_nn = []
rouge_l_list_nn = []

rouge_1_list_tf_idf = []
rouge_2_list_tf_idf = []
rouge_l_list_tf_idf = []

article_types = ['business', 'entertainment', 'politics', 'sport', 'tech']

for article_type in article_types:
    for i in range(101, 380):
#         print(i)
        article_file = io.open("train_original/" + article_type + "/article (" + str(i) +").txt", "r", encoding='utf8')
        article_file.readline()
        article = article_file.read()
        article_preprocessed = preprocessing(article)
        article_file.close()

        summarized_file = io.open("train_summary/" + article_type + "/summary (" + str(i) +").txt", "r", encoding='utf8')
        summarized = summarized_file.read()
        summarized_file.close()
        original_sentences = sent_tokenize(article)
        original_sentences[0] = original_sentences[0][1:] # to remove the \n
        rouge_1_nn, rouge_2_nn, rouge_l_nn, rouge_1_tf_idf, rouge_2_tf_idf, rouge_l_tf_idf = test(
            article_preprocessed, article, summarized, 0.35, original_sentences, i)

        rouge_1_list_nn.append(rouge_1_nn)
        rouge_2_list_nn.append(rouge_2_nn)
        rouge_l_list_nn.append(rouge_l_nn)

        rouge_1_list_tf_idf.append(rouge_1_tf_idf)
        rouge_2_list_tf_idf.append(rouge_2_tf_idf)
        rouge_l_list_tf_idf.append(rouge_l_tf_idf)


print('Using nn')
print('Average Rouge 1 score is: %f' % (np.average(rouge_1_list_nn)))
print('Average Rouge 2 score is: %f' % (np.average(rouge_2_list_nn)))
print('Average Rouge l score is: %f' % (np.average(rouge_l_list_nn)))

print('Using tf_idf only')
print('Average Rouge 1 score is: %f' % (np.average(rouge_1_list_tf_idf)))
print('Average Rouge 2 score is: %f' % (np.average(rouge_2_list_tf_idf)))
print('Average Rouge l score is: %f' % (np.average(rouge_l_list_tf_idf)))

Using nn
Average Rouge 1 score is: 0.768316
Average Rouge 2 score is: 0.694010
Average Rouge l score is: 0.764305
Using tf_idf only
Average Rouge 1 score is: 0.376493
Average Rouge 2 score is: 0.211553
Average Rouge l score is: 0.356842


In [22]:
# testing on cnn dailymail dataset
rouge_1_list_nn = []
rouge_2_list_nn = []
rouge_l_list_nn = []

rouge_1_list_tf_idf = []
rouge_2_list_tf_idf = []
rouge_l_list_tf_idf = []

df = pd.read_csv('test.csv')
articles_cnn = df['article']
summaries_cnn = df['highlights']

i = 0

for i in range (len(articles_cnn)):
    article_cnn_preprocessed = preprocessing(articles_cnn[i])
    original_sentences = sent_tokenize(articles_cnn[i])
    rouge_1_nn, rouge_2_nn, rouge_l_nn, rouge_1_tf_idf, rouge_2_tf_idf, rouge_l_tf_idf = test(
        article_cnn_preprocessed, articles_cnn[i], summaries_cnn[i], 0.35, original_sentences, i)
    i += 1
    rouge_1_list_nn.append(rouge_1_nn)
    rouge_2_list_nn.append(rouge_2_nn)
    rouge_l_list_nn.append(rouge_l_nn)

    rouge_1_list_tf_idf.append(rouge_1_tf_idf)
    rouge_2_list_tf_idf.append(rouge_2_tf_idf)
    rouge_l_list_tf_idf.append(rouge_l_tf_idf)
    if (i == 1000):
        break



print('Using nn')
print('Average Rouge 1 score is: %f' % (np.average(rouge_1_list_nn)))
print('Average Rouge 2 score is: %f' % (np.average(rouge_2_list_nn)))
print('Average Rouge l score is: %f' % (np.average(rouge_l_list_nn)))

print('Using tf_idf only')
print('Average Rouge 1 score is: %f' % (np.average(rouge_1_list_tf_idf)))
print('Average Rouge 2 score is: %f' % (np.average(rouge_2_list_tf_idf)))
print('Average Rouge l score is: %f' % (np.average(rouge_l_list_tf_idf)))

    

Using nn
Average Rouge 1 score is: 0.287436
Average Rouge 2 score is: 0.114622
Average Rouge l score is: 0.272895
Using tf_idf only
Average Rouge 1 score is: 0.196080
Average Rouge 2 score is: 0.053615
Average Rouge l score is: 0.181958


In [23]:
def extractive_summary(article, compression_ratio):
    original_sentences = sent_tokenize(article)
    article_preprocessed_entered = preprocessing(article)
    X_test_entered = generate_X_labels(article_preprocessed_entered, original_sentences)
    summary_predicted = predict_extractive_model(model, X_test_entered)
    num_sentences_summarized = math.ceil(compression_ratio * len(original_sentences))
    
    
    highest = np.argsort(summary_predicted[0]) [::-1]
    highest = highest[: num_sentences_summarized]
    highest = sorted(highest) # uncomment to arrange the article
    output_sentences = []
    
    for i in range (0, num_sentences_summarized):
        output_sentences.append(original_sentences[highest[i]])
        
    
    output_sentences = ' '.join(output_sentences)
    
    return output_sentences
    

In [24]:
article_file = io.open("cr7.txt", "r", encoding='utf8')
article_file.readline()
article = article_file.read()
article_file.close
summary = extractive_summary(article, 0.35)
print(summary)


Ronaldo picked up the award for the second year in a row to equal the record of Barcelona star Lionel Messi, and he said he does not believe any player is better than him. He told France Football (h/t Goal's Robin Bairner): "I've never seen anyone better than me. No footballer can do the things I can. "There’s no player more complete than me. But I tell you: there’s no one more complete than me. Legends like Floyd Mayweather [Jr.] and LeBron James don’t get to their perfect level by chance.


In [25]:
article_file = io.open("godzilla.txt", "r", encoding='utf8')
article = article_file.read()
article_file.close
summary = extractive_summary(article, 0.35)
print(summary)

An actor dressed as the giant creature breathed smoke over photographers on Monday as Godzilla received the 2,271st star on Hollywood Boulevard. "Godzilla should thank you for this historical and monumental star," said Final Wars producer Shogo Tomiyama. Hollywood's honorary mayor, Johnny Grant, said: "I do hereby proclaim this Godzilla Day in Hollywood. The premiere of Godzilla: Final Wars at Grauman's Chinese Theatre followed the ceremony on Hollywood Boulevard. Director Ryuhei Kitamura said it may not be Godzilla's final outing, as it has been billed. And producer Shogo Tomiyama added: "So long as Godzilla can fascinate people, I believe he will be resurrected by new generations of filmmakers in the future."


# Abstractive Summary

In [2]:
train_fn = trax.data.TFDS('cnn_dailymail', data_dir='abstractive_summary_data/', keys=('article', 'highlights'), train=True)

eval_fn = trax.data.TFDS('cnn_dailymail', data_dir='abstractive_summary_data/', keys=('article', 'highlights'), train=False)

/home/mohammedzaki/anaconda3/lib/python3.10/site-packages/jax/_src/xla_bridge.py:658: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  warnings.warn(
2023-06-03 01:58:33.413458: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [7]:
VOCAB_DIR = 'abstractive_summary_data/'
VOCAB_FILE = 'summarize32k.subword.subwords'
MODEL_DIR = 'abstractive_summary_data/pretrained_model'
FINE_TUNED_DIR = 'abstractive_summary_data/fine_tuning_weights'

In [2]:
def tokenize(words):
    tokens =  next(trax.data.tokenize(iter([words]), vocab_dir = VOCAB_DIR, vocab_file = VOCAB_FILE))
    tokens = list(tokens)
    tokens.append(1)
    return tokens

In [3]:
def detokenize(tokens):
    words = trax.data.detokenize(tokens, vocab_dir = VOCAB_DIR, vocab_file = VOCAB_FILE)
    words = wrapper.fill(words)
    return words

In [6]:
def preprocess_abstractive(stream):
    for (article, summary) in stream:
        combined = np.array(list(article) + [1, 0] + list(summary) + [1])
        mask = [0] * (len(list(article)) + 2) + [1] * (len(list(summary)) + 1)
        yield combined, combined, np.array(mask)

input_pipeline = trax.data.Serial(trax.data.Tokenize(vocab_dir = VOCAB_DIR, vocab_file = VOCAB_FILE), preprocess_abstractive, trax.data.FilterByLength(4096))

train_stream = input_pipeline(train_fn())
eval_stream = input_pipeline(eval_fn())

In [7]:
boundaries =  [128, 256,  512, 1024]
batch_sizes = [16,    8,    4,    2, 1]

# Create the streams.
train_batch_stream = trax.data.BucketByLength(boundaries, batch_sizes)(train_stream)

eval_batch_stream = trax.data.BucketByLength(boundaries, batch_sizes)(eval_stream)

In [4]:
def DecoderBlock(model_dim, feed_forward_depth, heads_num, activation_function, mode, dropout):
    features_dim = model_dim
    heads_dim = features_dim // heads_num

    # X is an input of dimension (batch_size, seqlen, heads_num x heads_dim) and will be converted by the following line to 
    # (batch_size x heads_num, seqlen, heads_dim) to allow matrix multiplication 

    compute_wq_wk_wv = tlayer.Fn('AttnHeads', 
                                      lambda x: trax_np.reshape(trax_np.transpose(
                                          trax_np.reshape(x, (x.shape[0], x.shape[1], heads_num, heads_dim)),
                                          (0, 2, 1, 3)),
                                          (-1, x.shape[1], heads_dim)), n_out=1)
    

    # Create feed-forward block (list) with two dense layers with dropout and input normalized
    # Add list of two Residual blocks: the attention with normalization and dropout and feed-forward blocks
    return [
      tlayer.Residual(
          tlayer.LayerNorm(),
          tlayer.Serial(
            tlayer.Branch( # creates three towers for one input, takes activations and creates queries keys and values
                [tlayer.Dense(features_dim), compute_wq_wk_wv], # queries
                [tlayer.Dense(features_dim), compute_wq_wk_wv], # keys
                [tlayer.Dense(features_dim), compute_wq_wk_wv], # values
            ),
            tlayer.Fn('DotProductAttn', lambda query, key, value: trax_np.matmul(trax_np.exp(trax_np.where(
                                trax_np.tril(trax_np.ones((1, query.shape[-2], query.shape[-2]), 
                                dtype=trax_np.bool_), k=0), trax_np.matmul(query, trax_np.swapaxes(key, -1, -2)) / trax_np.sqrt(query.shape[-1]),
                                trax_np.full_like(trax_np.matmul(query, trax_np.swapaxes(key, -1, -2)) / trax_np.sqrt(query.shape[-1]), -1e9)) -                      
                                trax.fastmath.logsumexp(trax_np.where(trax_np.tril(trax_np.ones((1, query.shape[-2], query.shape[-2]), dtype=trax_np.bool_), k=0),
                                trax_np.matmul(query, trax_np.swapaxes(key, -1, -2)) / trax_np.sqrt(query.shape[-1]),
                                trax_np.full_like(trax_np.matmul(query, trax_np.swapaxes(key, -1, -2)) / trax_np.sqrt(query.shape[-1]), -1e9)),
                                axis=-1, keepdims=True)), value), n_out=1), 
            tlayer.Fn('AttnOutput', lambda x: trax_np.reshape(trax_np.transpose(
                                trax_np.reshape(x, ( -1, heads_num, x.shape[1], heads_dim)), 
                                ( 0, 2, 1 , 3)), (-1, x.shape[1], heads_num * heads_dim)), n_out=1), # to allow for parallel


            tlayer.Dense(features_dim) # Final dense layer
            ),
          tlayer.Dropout(rate = dropout, mode = mode)
        ),
      tlayer.Residual(
            [ 
                tlayer.LayerNorm(),
                tlayer.Dense(feed_forward_depth),
                activation_function(), # Generally ReLU
                tlayer.Dropout(rate = dropout, mode = mode),
                tlayer.Dense(model_dim),
                tlayer.Dropout(rate = dropout,mode = mode)
            ]
        )
      ]

In [5]:
def TransformerLM(vocab_size=33300, model_dim=512, feed_forward_depth=2048,
                  layers_num=6, heads_num=8, dropout=0.1, max_len=4096,
                  mode='train', activation_function=tlayer.Relu):

    positional_encoder = [ 
        tlayer.Embedding(vocab_size, model_dim),
        tlayer.Dropout(rate=dropout, mode=mode),
        tlayer.PositionalEncoding(max_len=max_len, mode=mode)]

    decoder_blocks = [ 
        DecoderBlock(model_dim, feed_forward_depth, heads_num, activation_function, mode, dropout) 
        for _ in range(layers_num)]

    return tlayer.Serial(
        tlayer.ShiftRight(mode=mode),
        positional_encoder,
        decoder_blocks,
        tlayer.LayerNorm(),
        tlayer.Dense(vocab_size),
        tlayer.LogSoftmax()
    )

In [10]:
# fine tuning the pretrained model 

abstractive_model = TransformerLM()

# Load the pre-trained weights
abstractive_model.init_from_file(MODEL_DIR + '/model.pkl.gz', weights_only=True)

train_task = training.TrainTask( 
    labeled_data=train_batch_stream, # The training generator
    loss_layer=tlayer.CrossEntropyLoss(), # Loss function 
    optimizer=trax.optimizers.Adam(0.0000001),
    n_steps_per_checkpoint=5
)

# Define evaluation loop
eval_task = training.EvalTask(
    labeled_data = eval_batch_stream,
    metrics=[tlayer.CrossEntropyLoss(), tlayer.Accuracy()],
)

In [ ]:
loop = training.Loop(abstractive_model, train_task, eval_tasks = [eval_task], output_dir = FINE_TUNED_DIR)
loop.run(100)

/home/mohammedzaki/anaconda3/lib/python3.10/site-packages/trax/layers/base.py:851: FutureWarning: GzipFile was opened for writing, but this will change in future Python releases.  Specify the mode argument for opening it for writing.
  with gzip.GzipFile(fileobj=f, compresslevel=compresslevel) as gzipf:



Step      1: Total number of trainable weights: 55144980
Step      1: Ran 1 train steps in 66.42 secs
Step      1: train CrossEntropyLoss |  3.92982721


/home/mohammedzaki/anaconda3/lib/python3.10/site-packages/trax/supervised/training.py:1249: FutureWarning: GzipFile was opened for writing, but this will change in future Python releases.  Specify the mode argument for opening it for writing.
  with gzip_lib.GzipFile(fileobj=f, compresslevel=2) as gzipf:


Step      1: eval  CrossEntropyLoss |  4.05336428
Step      1: eval          Accuracy |  0.54263568

Step      5: Ran 4 train steps in 161.34 secs
Step      5: train CrossEntropyLoss |  3.19703913
Step      5: eval  CrossEntropyLoss |  4.02799606
Step      5: eval          Accuracy |  0.53191489

Step     10: Ran 5 train steps in 164.63 secs
Step     10: train CrossEntropyLoss |  3.01276636
Step     10: eval  CrossEntropyLoss |  3.62366676
Step     10: eval          Accuracy |  0.51886791

Step     15: Ran 5 train steps in 175.24 secs
Step     15: train CrossEntropyLoss |  3.29948545
Step     15: eval  CrossEntropyLoss |  5.08440685
Step     15: eval          Accuracy |  0.42105263

Step     20: Ran 5 train steps in 128.95 secs
Step     20: train CrossEntropyLoss |  3.45422411
Step     20: eval  CrossEntropyLoss |  4.29900599
Step     20: eval          Accuracy |  0.36563876

Step     25: Ran 5 train steps in 166.01 secs
Step     25: train CrossEntropyLoss |  3.22532773
Step     25: ev

In [8]:
# Get the model architecture
pretrained_model = TransformerLM(mode='eval')

# Load the pre-trained weights
pretrained_model.init_from_file(MODEL_DIR + '/model.pkl.gz', weights_only=True)

((((), (), ()),
  array([[ 0.00185256, -0.00332041, -0.00516087, ...,  0.02195336,
           0.01051251, -0.00567034],
         [-0.00739519,  0.00183287,  0.00196656, ...,  0.05615862,
           0.00132284, -0.00291792],
         [ 0.01545155,  0.00096676, -0.00778854, ..., -0.01111389,
           0.01328059, -0.00775655],
         ...,
         [ 0.01310026,  0.0053312 , -0.00110446, ...,  0.0018376 ,
          -0.01304161,  0.01059895],
         [ 0.01033204,  0.00199992, -0.00960302, ..., -0.00053245,
          -0.00377625,  0.00046856],
         [ 0.00082984, -0.00329287, -0.00404815, ...,  0.00028511,
          -0.00377798, -0.00656456]], dtype=float32),
  (),
  array([[ 0.01798599,  0.00387794,  0.00273774, ..., -0.07271248,
           0.01859818,  0.0302424 ],
         [ 0.05207964,  0.02332034,  0.00967251, ..., -0.02486727,
           0.0165352 , -0.00352491],
         [ 0.03194407, -0.02127359, -0.00068408, ..., -0.01532798,
           0.01319801, -0.00402305],
         ..

In [9]:
# Get the model architecture
abstractive_fine_tuned_model = TransformerLM(mode='eval')

# Load the pre-trained weights
abstractive_fine_tuned_model.init_from_file(FINE_TUNED_DIR + '/model.pkl.gz', weights_only=True)

((((), (), ()),
  array([[ 0.00185314, -0.00331989, -0.00516132, ...,  0.02194645,
           0.0105072 , -0.00566858],
         [-0.00739478,  0.00183274,  0.00196596, ...,  0.05613796,
           0.0013204 , -0.00291684],
         [ 0.01544436,  0.0009663 , -0.00778807, ..., -0.01110793,
           0.01327326, -0.0077543 ],
         ...,
         [ 0.013095  ,  0.00532906, -0.00110402, ...,  0.00183687,
          -0.0130364 ,  0.0105947 ],
         [ 0.0103279 ,  0.00199912, -0.00959918, ..., -0.00053224,
          -0.00377475,  0.00046837],
         [ 0.00082951, -0.00329155, -0.00404653, ...,  0.000285  ,
          -0.00377647, -0.00656193]], dtype=float32),
  (),
  array([[ 0.01797621,  0.00387489,  0.00273348, ..., -0.07268105,
           0.01858758,  0.0302281 ],
         [ 0.05205658,  0.02330974,  0.00966574, ..., -0.0248561 ,
           0.01652611, -0.00352417],
         [ 0.03192862, -0.02126632, -0.00068625, ..., -0.01532226,
           0.01319057, -0.00402066],
         ..

In [10]:
def abstractive_summary(article, model):
    article_summary = tokenize(article)
    article_summary.append(0)
    summary = []
    generated_word = 0
    start_summary = len(article_summary) 
    while generated_word != 1:
        length_padding = np.power(2, int(np.ceil(np.log2(len(article_summary) + 1)))) - len(article_summary)
        article_summary_padded = article_summary.copy()
        for _ in range(length_padding):
            article_summary_padded.append(0)
        article_summary_padded = np.array(article_summary_padded)[None, :]
        output, _ = model((article_summary_padded, article_summary_padded))  
        log_probs = output[0, len(article_summary), :]
        generated_word = int(np.argmax(log_probs))
        article_summary.append(generated_word)
        
    for i in range(start_summary, len(article_summary)):
        summary.append(article_summary[i])
        
    summary = detokenize(summary[: -1])
    return summary


In [11]:
test_sentence = "It was a sunny day when I went to the market to buy some flowers. But I only found roses, not tulips."
print(wrapper.fill(test_sentence), '\n')
print(abstractive_summary(test_sentence, abstractive_fine_tuned_model))

It was a sunny day when I went to the market to buy some flowers. But
I only found roses, not tulips. 

: I just found roses, not tulips.


In [12]:
article = "It’s the posing craze sweeping the U.S. after being brought to fame by skier Lindsey Vonn, soccer star Omar Cummings, baseball player Albert Pujols - and even Republican politician Rick Perry. But now four students at Riverhead High School on Long Island, New York, have been suspended for dropping to a knee and taking up a prayer pose to mimic Denver Broncos quarterback Tim Tebow. Jordan Fulcoly, Wayne Drexel, Tyler Carroll and Connor Carroll were all suspended for one day because the ‘Tebowing’ craze was blocking the hallway and presenting a safety hazard to students. Scroll down for video. Banned: Jordan Fulcoly, Wayne Drexel, Tyler Carroll and Connor Carroll (all pictured left) were all suspended for one day by Riverhead High School on Long Island, New York, for their tribute to Broncos quarterback Tim Tebow. Issue: Four of the pupils were suspended for one day because they allegedly did not heed to warnings that the 'Tebowing' craze at the school was blocking the hallway and presenting a safety hazard to students."
print(wrapper.fill(article), '\n')
print(abstractive_summary(article, abstractive_fine_tuned_model))

It’s the posing craze sweeping the U.S. after being brought to fame by
skier Lindsey Vonn, soccer star Omar Cummings, baseball player Albert
Pujols - and even Republican politician Rick Perry. But now four
students at Riverhead High School on Long Island, New York, have been
suspended for dropping to a knee and taking up a prayer pose to mimic
Denver Broncos quarterback Tim Tebow. Jordan Fulcoly, Wayne Drexel,
Tyler Carroll and Connor Carroll were all suspended for one day
because the ‘Tebowing’ craze was blocking the hallway and presenting a
safety hazard to students. Scroll down for video. Banned: Jordan
Fulcoly, Wayne Drexel, Tyler Carroll and Connor Carroll (all pictured
left) were all suspended for one day by Riverhead High School on Long
Island, New York, for their tribute to Broncos quarterback Tim Tebow.
Issue: Four of the pupils were suspended for one day because they
allegedly did not heed to warnings that the 'Tebowing' craze at the
school was blocking the hallway and presen

In [12]:
def test_abstractive(original_test, summarized_text, fine_tuned_model, pretrained_model):
    summary_fine_tuned_model = abstractive_summary(original_test, fine_tuned_model)
    summary_pretrained_model = abstractive_summary(original_test, pretrained_model)
    print(summary_fine_tuned_model)
    print('--------------------------')
    
    print(summary_pretrained_model)
    print('-----------------------------------------------------------')
    rouge = Rouge()
    scores_transformer_fine_tuned_model = rouge.get_scores(summary_fine_tuned_model, summarized_text)
    scores_transformer_fine_tuned_model = scores_transformer_fine_tuned_model[0]
    
    rouge_1_transformer_fine_tuned_model = scores_transformer_fine_tuned_model['rouge-1']['f']
    rouge_2_transformer_fine_tuned_model = scores_transformer_fine_tuned_model['rouge-2']['f']
    rouge_l_transformer_fine_tuned_model = scores_transformer_fine_tuned_model['rouge-l']['f']
    
    scores_transformer_pretrained_model = rouge.get_scores(summary_pretrained_model, summarized_text)
    scores_transformer_pretrained_model = scores_transformer_pretrained_model[0]
    
    rouge_1_transformer_pretrained_model = scores_transformer_pretrained_model['rouge-1']['f']
    rouge_2_transformer_pretrained_model = scores_transformer_pretrained_model['rouge-2']['f']
    rouge_l_transformer_pretrained_model = scores_transformer_pretrained_model['rouge-l']['f']
    
    return rouge_1_transformer_fine_tuned_model, rouge_2_transformer_fine_tuned_model, rouge_l_transformer_fine_tuned_model, rouge_1_transformer_pretrained_model, rouge_2_transformer_pretrained_model, rouge_l_transformer_pretrained_model

In [13]:
# testing on BBC news summary

rouge_1_list_transformer = []
rouge_2_list_transformer = []
rouge_l_list_transformer = []

rouge_1_list_transformer_pretrained = []
rouge_2_list_transformer_pretrained = []
rouge_l_list_transformer_pretrained = []

article_types = ['business', 'entertainment', 'politics', 'sport', 'tech']

for article_type in article_types:
    for i in range(101, 121):

        article_file = io.open("train_original/" + article_type + "/article (" + str(i) +").txt", "r", encoding='utf8')
        article_file.readline()
        article = article_file.read()
        article_file.close()

        summarized_file = io.open("train_summary/" + article_type + "/summary (" + str(i) +").txt", "r", encoding='utf8')
        summarized = summarized_file.read()
        summarized_file.close()

        rouge_1_transformer, rouge_2_transformer, rouge_l_transformer, rouge_1_pretrained, rouge_2_pretrained, rouge_l_pretrained= test_abstractive(article, summarized, abstractive_fine_tuned_model, pretrained_model)

        rouge_1_list_transformer.append(rouge_1_transformer)
        rouge_2_list_transformer.append(rouge_2_transformer)
        rouge_l_list_transformer.append(rouge_l_transformer)
        rouge_1_list_transformer_pretrained.append(rouge_1_pretrained)
        rouge_2_list_transformer_pretrained.append(rouge_2_pretrained)
        rouge_l_list_transformer_pretrained.append(rouge_l_pretrained)


print('Using transformer')
print('Average Rouge 1 score is: %f' % (np.average(rouge_1_list_transformer)))
print('Average Rouge 2 score is: %f' % (np.average(rouge_2_list_transformer)))
print('Average Rouge l score is: %f' % (np.average(rouge_l_list_transformer)))
print('Average Rouge 1 score is: %f' % (np.average(rouge_1_list_transformer_pretrained)))
print('Average Rouge 2 score is: %f' % (np.average(rouge_2_list_transformer_pretrained)))
print('Average Rouge l score is: %f' % (np.average(rouge_l_list_transformer_pretrained)))

The move was criticised by some analysts and 10st figures. The move
was criticised by some analysts and 10st figures. The move was
criticised by some analysts.10; The rate hike was the first since
2003.
--------------------------
The move was criticised by some analysts and some analysts. It was
criticised by some analysts and some analysts. The move was ill-timed
given data showing the economy grew just 0.1% between October and
December.
-----------------------------------------------------------
The Titan corporation was accused of funnelling more than $2m into the
2001 re-election campaign of President Mathieu Kerekou. The company,
which has agreed to pay $28.5m in criminal penalties, was accused of
funnelling more than $2m into the 2001 re-election campaign of
President Mathieu Kerekou. The company says it no longer tolerates
such practices.
--------------------------
The Titan corporation was accused of funnelling more than $2m into the
2001 re-election campaign of President Mathi

Icl and Wellbutrin to a weak dollar hit profits, but global sales were
up 1% in 2004. Company says it had absorbed over £1.5bn of lost sales
to generics but still managing to grow the business.
--------------------------
Icl and Wellbutrin to a weak dollar hit profits, but global sales were
up 1% in 2004. Company says it had absorbed over £1.5bn of lost sales
to generics but still managing to grow the business.
-----------------------------------------------------------
SMFG, and Daiwa Securities jumped amid speculation that two of Japan's
biggest financial companies will merge. The merger would be a good
move, he added.
--------------------------
SMFG, and Daiwa Securities jumped amid speculation that two of Japan's
biggest financial companies will merge. The merger "would be a good
move," he added.
-----------------------------------------------------------
German-US carmaker saw fourth quarter operating profits fall to 785m
euros. Firm said it was determined to retain Mercedes' posi

Police arrested Marion "Suge" Knight for allegedly marijuana
possession. He was arrested in Barstow, California, on Saturday
following an alleged traffic offence. He was arrested on Saturday
after performing an illegal U-turn and a search of his car allegedly
found marijuana.
--------------------------
Police arrested Marion "Suge" Knight for allegedly marijuana
possession. He was arrested in Barstow, California, on Saturday
following an alleged traffic offence. He was arrested on Saturday
after performing an illegal U-turn and a search of his car allegedly
found marijuana.
-----------------------------------------------------------
Sir Elton played a 2,700-strong audience on Sunday at the French
capital's Bastille opera house. The singer played piano accompaniment
throughout the concert. The singer also played a 2,700-strong audience
on Sunday at the French capital's Bastille opera house.
--------------------------
Sir Elton played a 2,700-strong audience on Sunday at the French
capit

Home Office minister said 'UK Muslims should accept that people of
Islamic appearance are more likely to be stopped and searched by
police. She said Muslim groups would be asked what other
qualifications and skills. Comes after Muslim groups have repeatedly
claimed their communities are being victimised under terror laws.
--------------------------
Home Office minister said 'UK Muslims should accept that people of
Islamic appearance are more likely to be stopped and searched by
police. She said Muslim groups would be asked what other
qualifications and skills. Comes after Muslim groups have repeatedly
claimed their communities are being victimised under terror laws.
-----------------------------------------------------------
Tory leader says English local authorities spent more than £3bn. Mr
Howard is expected to tell activists in Kent voters' tolerance. Other
parties and refugee agencies have already attacked Tory plans for
annual limit on numbers.
--------------------------
Tory lead

Lord Scarman was one of the "great advocates of our generation" He
also chaired the 1969 tribunal set up to investigate civil
disturbances in Northern Ireland. Lord Scarman was one of the "great
advocates of our generation" He also chaired the 1969 tribunal set up
to investigate civil disturbances in Northern Ireland.
--------------------------
Lord Scarman conducted the inquiry into the 1981 Brixton riots. He was
one of the "great advocates of our generation" He was one of the
"great advocates of our generation" Lord Scarman died aged 93 from
93.10sec .
-----------------------------------------------------------
Prince Harry was in a privileged position and said he should
apologise. Spoke to the palace after he said he should apologise in
person. Spoke to the palace after the Duchess of Cambridge's comments.
--------------------------
Prince Harry was in a privileged position and said he should
apologise. Spoke to the palace after he said he should apologise in
person. Spoke to the pa

Shaun Wright-Phillips will start against the Netherlands on the left.
Eriksson also revealed that Wes Brown will start alongside Jamie
Carragher. Andy Johnson and Stewart Downing also get a chance to
impress.
--------------------------
Shaun Wright-Phillips will start against the Netherlands on the left.
Eriksson also revealed that Wes Brown will start alongside Jamie
Carragher. Andy Johnson and Stewart Downing will start for the game.
-----------------------------------------------------------
Arjen Robben sustained a broken metatarsal in the Premiership win at
Blackburn. The 21-year-old missed the first three months of the season
with a similar injury. The 21-year-old missed the first three months
of the season with a similar injury.
--------------------------
Arjen Robben sustained a broken metatarsal in the Premiership win at
Blackburn. The 21-year-old missed the first three months of the season
with a similar injury. The 21-year-old missed the first three months
of the season with

Indian writer Rohit Gupta edits a group blog without Borders. He
created the blogger's blog without Borders. He was using SMS text
messages from affected areas of Sri Lanka. Mr Gupta and his fellow
bloggers switched gears.
--------------------------
Indian writer Rohit Gupta edits a group blog without Borders. He
created the blogger's blog without Borders. He was using SMS text
messages from affected areas of Sri Lanka. Mr Gupta and fellow
bloggers switched gears.
-----------------------------------------------------------
10; The Wolong Nature Reserve in the Sichuan Province of southwest
China is home to 20% of the remaining 1,500 giant pandas in the world.
The network has been developed by Intel, working closely with the
staff at Wolong. It includes a 802.11b wireless network and a video
monitoring system using five cameras to observe pandas around the
clock.
--------------------------
10st panda population is getting a helping hand from wireless internet
network. The network has bee

Suspect arrested in Camberley, Surrey, on Wednesday. He has now been
bailed to return to a Surrey police station in December. All 28 people
detained globally are suspected of being involved in an internet-based
network. Police in the US, who have indicted 19 people in Newark, New
Jersey, New Jersey, New Jersey.
--------------------------
Suspect arrested in Camberley, Surrey, on Wednesday. He has now been
bailed to return to a Surrey police station in December. All 28 people
detained globally are suspected of being involved in an internet-based
network. Police say the fraud caused losses of more than $4m.
-----------------------------------------------------------
Using transformer
Average Rouge 1 score is: 0.331689
Average Rouge 2 score is: 0.201642
Average Rouge l score is: 0.323505
Average Rouge 1 score is: 0.318883
Average Rouge 2 score is: 0.193023
Average Rouge l score is: 0.311299


In [17]:
# testing on cnn dailymail dataset

rouge_1_list_transformer = []
rouge_2_list_transformer = []
rouge_l_list_transformer = []

df = pd.read_csv('test.csv')
articles_cnn = df['article']
summaries_cnn = df['highlights']

i = 0
for article_cnn, summary_cnn in zip(articles_cnn, summaries_cnn):
    rouge_1_transformer, rouge_2_transformer, rouge_l_transformer, rouge_1_pretrained, rouge_2_pretrained, rouge_l_pretrained= test_abstractive(article_cnn, summary_cnn, abstractive_fine_tuned_model, pretrained_model)
    i += 1
    if (i == 30):
        break
    
    rouge_1_list_transformer.append(rouge_1_transformer)
    rouge_2_list_transformer.append(rouge_2_transformer)
    rouge_l_list_transformer.append(rouge_l_transformer)
    rouge_1_list_transformer_pretrained.append(rouge_1_pretrained)
    rouge_2_list_transformer_pretrained.append(rouge_2_pretrained)
    rouge_l_list_transformer_pretrained.append(rouge_l_pretrained)


print('Using transformer')
print('Average Rouge 1 score is: %f' % (np.average(rouge_1_list_transformer)))
print('Average Rouge 2 score is: %f' % (np.average(rouge_2_list_transformer)))
print('Average Rouge l score is: %f' % (np.average(rouge_l_list_transformer)))
print('Average Rouge 1 score is: %f' % (np.average(rouge_1_list_transformer_pretrained)))
print('Average Rouge 2 score is: %f' % (np.average(rouge_2_list_transformer_pretrained)))
print('Average Rouge l score is: %f' % (np.average(rouge_l_list_transformer_pretrained)))

A U.S consumer advisory group set up by the Department of
Transportation. The government is happy to set standards for animals
flying on planes. But some experts are questioning whether planes are
putting passengers at risk.
--------------------------
A U.S consumer advisory group set up by the Department of
Transportation. The government is happy to set standards for flying on
planes. But the DOT and FAA take a stand for humane treatment of
passengers.
-----------------------------------------------------------
Rahul Kumar, 17, climbed into the lions' enclosure at a zoo in
Ahmedabad. He was drunk and 'thought I'd stand a good chance' He was
sitting near the enclosure when he made a dash for the lions, adding:
'Guards had earlier spotted him close to the enclosure'
--------------------------
Rahul Kumar, 17, climbed into the lions' enclosure at a zoo in
Ahmedabad. He was drunk and 'thought I'd stand a good chance' He was
sitting near the enclosure when he made a dash for the lions, add

Ian Bell was on the second day of England's opening tour match. Ian
Bell was on the second day of England's opening tour match. Ian Bell
was on the second day of the St Kitts and Nevis Invitational XI vs
England tour match .
--------------------------
Ian Bell was on the second day of England's opening tour match. Ian
Bell was on the second day of England's opening tour match. Ian Bell
was on the second day of the St Kitts and Nevis Invitational XI vs
England tour match .
-----------------------------------------------------------
A senior U.S. official says the U.S. has made no serious effort to
negotiate for the 73-year-old development expert's release. The
Pakistani official says the capture by al Qaeda made it hard for the
U.S. to negotiate. A senior Pakistani official says the capture by al
Qaeda made it hard for the U.S. to negotiate.
--------------------------
A senior U.S. official says the U.S. has made no serious effort to
negotiate for the 73-year-old development expert's re

ISIS is a problem that is off the charts historically, he says. He
says the U.S. has sent the United States into "uncharted territory"
when it comes to the terror group. He says the U.S. has expanded its
reach to Libya, Egypt and Yemen.
--------------------------
ISIS is a problem that is off the charts historically, he says. He
says the U.S. has sent the United States into "uncharted territory"
when it comes to the terror group. He says the U.S. has expanded its
reach to Libya, Egypt and Yemen.
-----------------------------------------------------------
David Cameron has been secretly using hair dye to turn back the years
Nigel Farage suggested today. Mr Farage said he was 'jealous' of how
Tory leader has reversed the greying process.
--------------------------
David Cameron has been secretly using hair dye to turn back the years
Nigel Farage. Mr Farage said he was 'jealous' of how Tory leader has
reversed the greying process.
----------------------------------------------------------